ABRIR ARQUIVO TSP

In [ ]:
import matplotlib.pyplot as plt

# Função para ler o arquivo TSPLIB e extrair os pontos
def ler_arquivo_tsplib(nome_arquivo):
    pontos = []
    with open(nome_arquivo, "r") as arquivo:
        lendo_pontos = False
        for linha in arquivo:
            if "NODE_COORD_SECTION" in linha:
                lendo_pontos = True
                continue
            elif "EOF" in linha:
                break
            if lendo_pontos:
                partes = linha.strip().split()
                if len(partes) == 3:
                    _, x, y = partes
                    pontos.append((float(x), float(y)))
    return pontos

# Nome do arquivo TSPLIB
nome_arquivo = "arq1.tsp"

# Ler os pontos do arquivo
pontos = ler_arquivo_tsplib(nome_arquivo)

# Extrair as coordenadas x e y dos pontos
x = [ponto[0] for ponto in pontos]
y = [ponto[1] for ponto in pontos]

# Criar o gráfico de dispersão
plt.scatter(x, y, marker='o', color='blue')

# Definir rótulos para os pontos (opcional)
for i, ponto in enumerate(pontos):
    plt.annotate(str(i + 1), (x[i], y[i]))

# Definir título e rótulos dos eixos (opcional)
plt.title('Plot de Pontos TSPLIB')
plt.xlabel('Coordenada X')
plt.ylabel('Coordenada Y')

# Mostrar o gráfico
plt.show()

INICIAR ALGORITMO

In [ ]:
import random

def ler_arquivo_tsp(nome_arquivo):
    cidades = []
    with open(nome_arquivo, 'r') as arquivo:
        lendo_coord = False
        for linha in arquivo:
            linha = linha.strip()
            if linha.startswith("NODE_COORD_SECTION"):
                lendo_coord = True
            elif linha.startswith("EOF"):
                break
            elif lendo_coord:
                partes = linha.split()
                cidade_id = int(partes[0])
                x = float(partes[1])
                y = float(partes[2])
                cidades.append((cidade_id, x, y))
    return cidades

def criar_populacao_inicial(cidades, tamanho_populacao):
    # Crie indivíduos como permutações aleatórias das cidades
    populacao = []
    cidades_sem_origem = cidades[1:]  # Exclua a cidade de origem
    for _ in range(tamanho_populacao):
        cromossomo = random.sample(cidades_sem_origem, len(cidades_sem_origem))
        cromossomo.insert(0, cidades[0])  # Adicione a cidade de origem no início
        populacao.append(cromossomo)
    return populacao

# Substitua "exemplo.tsp" pelo nome do seu arquivo .tsp
nome_do_arquivo_tsp = "arq1.tsp"

# Leitura das coordenadas das cidades do arquivo .tsp
cidades = ler_arquivo_tsp(nome_do_arquivo_tsp)

# Inicialização da população
tamanho_populacao = 50  # Tamanho da população inicial
populacao_inicial = criar_populacao_inicial(cidades, tamanho_populacao)
print (populacao_inicial)

CALCULAR FITNESS

In [ ]:
def calcular_distancia_total(rota):
    distancia_total = 0.0
    for i in range(len(rota) - 1):
        cidade_atual = rota[i]
        proxima_cidade = rota[i + 1]
        distancia = calcular_distancia_entre_cidades(cidade_atual, proxima_cidade)
        distancia_total += distancia
    # Adicione a distância de volta à cidade de origem
    distancia_total += calcular_distancia_entre_cidades(rota[-1], rota[0])
    return distancia_total

def calcular_distancia_entre_cidades(cidade1, cidade2):
    x1, y1 = cidade1[1], cidade1[2]
    x2, y2 = cidade2[1], cidade2[2]
    # Use a fórmula da distância euclidiana para calcular a distância entre duas cidades
    distancia = ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5
    return distancia

# Função para calcular a aptidão de um cromossomo (rota)
def calcular_aptidao(cromossomo):
    distancia_total = calcular_distancia_total(cromossomo)
    return distancia_total

# Exemplo de uso:
for cromossomo in populacao_inicial:
    aptidao = calcular_aptidao(cromossomo)
    print("Aptidao da rota: {:.2f}".format(aptidao))

ALGORITMO GULOSO

In [ ]:
# Função para encontrar uma solução inicial usando um algoritmo guloso (greedy)
def algoritmo_guloso(cidades):
    cidade_atual = cidades[0]  # Comece na cidade de origem
    rota_gulosa = [cidade_atual]
    cidades_restantes = cidades[1:]

    while cidades_restantes:
        cidade_mais_proxima = min(cidades_restantes, key=lambda cidade: calcular_distancia_entre_cidades(cidade_atual, cidade))
        rota_gulosa.append(cidade_mais_proxima)
        cidade_atual = cidade_mais_proxima
        cidades_restantes.remove(cidade_mais_proxima)

    # Retorne à cidade de origem para completar o ciclo
    rota_gulosa.append(cidades[0])
   
    return rota_gulosa

# Substitua "arq1.tsp" pelo nome do seu arquivo .tsp
nome_do_arquivo_tsp = "arq1.tsp"

# Leitura das coordenadas das cidades do arquivo .tsp
cidades = ler_arquivo_tsp(nome_do_arquivo_tsp)

# Encontre uma solução inicial usando o algoritmo guloso
rota_gulosa = algoritmo_guloso(cidades)

# Exibir a rota encontrada pelo algoritmo guloso
x_guloso = [cidade[1] for cidade in rota_gulosa]
y_guloso = [cidade[2] for cidade in rota_gulosa]

plt.figure(figsize=(10, 6))
plt.scatter(x_guloso, y_guloso, marker='o', color='green', label='Rota Gulosa')
plt.plot(x_guloso + [x_guloso[0]], y_guloso + [y_guloso[0]], linestyle='-', color='purple')
plt.title('Rota Inicial Encontrada pelo Algoritmo Guloso')
plt.xlabel('Coordenada X')
plt.ylabel('Coordenada Y')
plt.legend()
plt.show()

SELEÇÃO

In [ ]:
import random
import matplotlib.pyplot as plt

# Função para ler o arquivo TSPLIB e extrair os pontos
def ler_arquivo_tsplib(nome_arquivo):
    pontos = []
    with open(nome_arquivo, "r") as arquivo:
        lendo_pontos = False
        for linha in arquivo:
            if "NODE_COORD_SECTION" in linha:
                lendo_pontos = True
                continue
            elif "EOF" in linha:
                break
            if lendo_pontos:
                partes = linha.strip().split()
                if len(partes) == 3:
                    _, x, y = partes
                    pontos.append((float(x), float(y)))
    return pontos

# Função para calcular a distância entre dois pontos
def calcular_distancia_entre_pontos(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5

# Função para calcular o comprimento da rota
def calcular_comprimento_rota(rota, pontos):
    comprimento = 0.0
    for i in range(len(rota) - 1):
        cidade_atual = rota[i]
        proxima_cidade = rota[i + 1]
        comprimento += calcular_distancia_entre_pontos(pontos[cidade_atual], pontos[proxima_cidade])
    # Adicione a distância de volta à cidade de origem
    comprimento += calcular_distancia_entre_pontos(pontos[rota[-1]], pontos[rota[0]])
    return comprimento

# Função para criar uma população inicial
def criar_populacao_inicial(num_cidades, tamanho_populacao):
    populacao = []
    for _ in range(tamanho_populacao):
        rota = list(range(num_cidades))
        random.shuffle(rota)
        populacao.append(rota)
    return populacao

# Função para realizar o crossover (cruzamento)
def crossover(pais):
    ponto_corte = random.randint(1, len(pais[0]) - 1)
    filho = pais[0][:ponto_corte] + [gene for gene in pais[1] if gene not in pais[0][:ponto_corte]]
    return filho

# Função para realizar a mutação (troca de dois genes)
def mutacao(rota):
    indice1, indice2 = random.sample(range(len(rota)), 2)
    rota[indice1], rota[indice2] = rota[indice2], rota[indice1]

# Função para executar o algoritmo genético
def algoritmo_genetico(num_cidades, tamanho_populacao, num_geracoes, nome_arquivo_tsp):
    # Ler os pontos do arquivo TSP
    pontos = ler_arquivo_tsplib(nome_arquivo_tsp)

    # Criar uma população inicial
    populacao = criar_populacao_inicial(num_cidades, tamanho_populacao)

    # Iniciar o processo de evolução
    melhor_rota = None
    melhor_comprimento = float('inf')
    melhores_comprimentos = []

    for geracao in range(num_geracoes):
        # Selecionar os pais por meio de um torneio
        pais = []
        for _ in range(tamanho_populacao // 2):
            participantes = random.sample(populacao, 4)  # Torneio com 4 indivíduos
            vencedor = min(participantes, key=lambda rota: calcular_comprimento_rota(rota, pontos))
            pais.append(vencedor)

        # Realizar o crossover e mutação para criar a próxima geração
        proxima_geracao = []
        while len(proxima_geracao) < tamanho_populacao:
            filho = crossover(random.choices(pais, k=2))
            mutacao(filho)
            proxima_geracao.append(filho)

        # Substituir a população antiga pela próxima geração
        populacao = proxima_geracao

        # Encontrar o melhor indivíduo desta geração
        for rota in populacao:
            comprimento = calcular_comprimento_rota(rota, pontos)
            if comprimento < melhor_comprimento:
                melhor_comprimento = comprimento
                melhor_rota = rota

        melhores_comprimentos.append(melhor_comprimento)

    return melhor_rota, melhor_comprimento, melhores_comprimentos

# Substitua "arq1.tsp" pelo nome do seu arquivo .tsp
nome_do_arquivo_tsp = "arq1.tsp"

# Parâmetros do algoritmo genético
num_cidades = len(ler_arquivo_tsplib(nome_do_arquivo_tsp))
tamanho_populacao = 500
num_geracoes = 2000

# Executar o algoritmo genético
melhor_rota, melhor_comprimento, melhores_comprimentos = algoritmo_genetico(num_cidades, tamanho_populacao, num_geracoes, nome_do_arquivo_tsp)

# Plotar o gráfico da melhor rota
melhor_x = [pontos[cidade][0] for cidade in melhor_rota]
melhor_y = [pontos[cidade][1] for cidade in melhor_rota]

plt.figure(figsize=(10, 6))
plt.scatter(x, y, marker='o', color='blue', label='Cidades')
plt.plot(melhor_x + [melhor_x[0]], melhor_y + [melhor_y[0]], linestyle='-', color='red', label='Melhor Rota')
plt.title('Melhor Rota Encontrada pelo Algoritmo Genético')
plt.xlabel('Coordenada X')
plt.ylabel('Coordenada Y')
plt.legend()
plt.show()

# Plotar o gráfico dos melhores comprimentos ao longo das gerações
plt.figure(figsize=(10, 6))
plt.plot(melhores_comprimentos, marker='o', color='green', linestyle='-', markersize=4)
plt.title('Melhores Comprimentos ao Longo das Gerações')
plt.xlabel('Geração')
plt.ylabel('Comprimento da Rota')
plt.grid(True)
plt.show()

print("Melhor Rota:", melhor_rota)
print("Melhor Comprimento:", melhor_comprimento)